# Geolocalisation of Planet data

In [138]:
# Setup code for the notebook
%matplotlib notebook
# Autoreload external python modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

from ipytools import display_imshow
from ipytools import readGTIFF
from ipytools import writeGTIFF
from ipytools import readGTIFFmeta
from ipytools import display_gallery
from ipytools import display_RSO

from glob import glob
from tqdm import tqdm
import json
import numpy as np

# RPC class
from rpc_model import RPCModel

# Tools to handle S03
from tools import s03_handler
from tools import data_handler
from tools import get_image_longlat_polygon
from tools import reprojection
from tools import geolocalisation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
lst_tif, area, triplets = s03_handler()

In [3]:
lst_tif

['../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0008.tif',
 '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0014.tif',
 '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d3_0014.tif',
 '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0002.tif',
 '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0018.tif',
 '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0010.tif',
 '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0002.tif',
 '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0005.tif',
 '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0017.tif',
 '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0011.tif',
 '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0004.tif',
 '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0004.tif',
 '..

In [4]:
area

{'d1': {1: {'1Z': '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0001.tif',
   '7Z': '../data/s03_20161003T161107Z/panchromatic/s03_20161003T161107Z_pan_d1_0001.tif',
   '8Z': '../data/s03_20161003T161148Z/panchromatic/s03_20161003T161148Z_pan_d1_0001.tif'},
  2: {'1Z': '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0002.tif',
   '7Z': '../data/s03_20161003T161107Z/panchromatic/s03_20161003T161107Z_pan_d1_0002.tif',
   '8Z': '../data/s03_20161003T161148Z/panchromatic/s03_20161003T161148Z_pan_d1_0002.tif'},
  3: {'1Z': '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0003.tif',
   '7Z': '../data/s03_20161003T161107Z/panchromatic/s03_20161003T161107Z_pan_d1_0003.tif',
   '8Z': '../data/s03_20161003T161148Z/panchromatic/s03_20161003T161148Z_pan_d1_0003.tif'},
  4: {'1Z': '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0004.tif',
   '7Z': '../data/s03_20161003T161107Z/panchromatic/s03_20161003T1611

In [5]:
triplets

{'1Z': {'d1': {1: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0001.tif',
   2: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0002.tif',
   3: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0003.tif',
   4: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0004.tif',
   5: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0005.tif',
   6: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0006.tif',
   7: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0007.tif',
   8: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0008.tif',
   9: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0009.tif',
   10: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0010.tif',
   11: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0011.tif',
   12: '../data/s0

# S03

In [6]:
from ipytools import clickablemap

In [33]:
# these lines create a map objet and display it
m = clickablemap(zoom=14)
display(m)

# we can move the map to any position by indicating its (latitude, longitude)
m.center = [35.96134549, -96.7190739]   # i.e. Cushing USA

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[48.790153, 2.327395], controls=(DrawControl(circle={'shapeOptions': {'color': '#0000FF'}}, layer=FeatureGroup(), polygon={'shapeOptions': {'color': '#0000FF'}}, polyline={'shapeOptions': {'color': '#0000FF'}}, rectangle={'shapeOptions': {'color': '#0000FF'}}),), layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']),), layout=Layout(height='500px', width='100%'), options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], scroll_wheel_zoom=True, zoom=14)

In [37]:
f_img = triplets['1Z']['d1'][1]
f_rpc = f_img.replace(".tif", "_rpc.txt")
f_rpc

'../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0001_rpc.txt'

In [38]:
rpc_f = RPCModel(triplets['1Z']['d1'][2].replace(".tif", "_rpc.txt"))
rpc_f


    # Projection function coefficients
      col_num = -0.0018  1.0424  0.0220 -0.0202 -0.0039  0.0033  0.0001 -0.0851 -0.0000 -0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
      col_den =  1.0000 -0.0824 -0.0011  0.0014 -0.0001  0.0000  0.0000  0.0001  0.0000 -0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
      row_num = -0.0015 -0.4367 -1.4931 -0.0566  0.2310  0.0088  0.0300  0.0337  0.3951  0.0006  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
      row_den =  1.0000 -0.0780 -0.2637 -0.0101  0.0001  0.0000  0.0000  0.0001 -0.0002  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000

    # Offsets and Scales
      row_offset = 674.5
      col_offset = 1599.5
      lat_offset = 36.05096386
      lon_offset = -96.72475481
      alt_offset = 282.0
      row_scale = 674.5
      col_scale = 1599.5
      lat_scale = 0.0092312
      lon_scal

In [25]:
rpc_f = RPCModel(triplets['1Z']['d1'][10].replace(".tif", "_rpc.txt"))
rpc_f


    # Projection function coefficients
      col_num = -0.0018  1.0424  0.0230 -0.0193 -0.0018  0.0012  0.0000 -0.0365  0.0000 -0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
      col_den =  1.0000 -0.0357  0.0001  0.0004 -0.0000  0.0000  0.0000  0.0000  0.0000 -0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
      row_num = -0.0016 -0.4502 -1.5096 -0.0596  0.1875  0.0074  0.0249  0.0279  0.3146  0.0005  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
      row_den =  1.0000 -0.0626 -0.2074 -0.0083  0.0001  0.0000  0.0000  0.0000 -0.0002  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000

    # Offsets and Scales
      row_offset = 674.5
      col_offset = 1599.5
      lat_offset = 35.96134549
      lon_offset = -96.7190739
      alt_offset = 282.0
      row_scale = 674.5
      col_scale = 1599.5
      lat_scale = 0.00966834
      lon_scal

In [97]:
mo = ipytools.overlaymap(footprint0, "tmp/d1_0001.png" , zoom = 13)
# mo.overlaymap(footprint1, "tmp/d1_0002.png")
display(mo)

VBox(children=(HBox(children=(Button(description='hide', layout=Layout(width='auto'), style=ButtonStyle()), Button(description='hidePoly', layout=Layout(width='auto'), style=ButtonStyle()), Label(value=''))), Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[36.0722810745507, -96.7442756453842], layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']), Polygon(fill_opacity=0.25, locations=[[36.0722810745507, -96.7442756453842], [36.0722810745507, -96.7093717643727], [36.0547109493463, -96.7093717643727], [36.0547109493463, -96.7442756453842]], options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'no_clip', 'opacity', 'pointer_events', 'smooth_factor', 'stroke', 'weight'], weight=2), ImageOverlay(bounds=[[36.0722810745507, -96.7093717643727], [36.0547109493463, -96.7442756453842]], options=['attribution'], url='tmp/d1_0001.png?05801')), layout=Layout(height='500px', width='100%'), options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], scroll_wheel_zoom=True, zoom=13)))

In [104]:
f_img

'../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0001.tif'

In [124]:
imgs = list(triplets['1Z']['d1'].values())
imgs.sort()
imgs

footprint = []
for i in imgs:
    footprint.append(get_image_longlat_polygon(i))
    
m = ipytools.clickablemap()

for i in range(len(imgs)):
    m.add_GeoJSON(footprint[i])

m.center = footprint0[ 'coordinates'][0][0][::-1]
display(m)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[36.0722810745507, -96.7442756453842], controls=(DrawControl(circle={'shapeOptions': {'color': '#0000FF'}}, layer=FeatureGroup(), polygon={'shapeOptions': {'color': '#0000FF'}}, polyline={'shapeOptions': {'color': '#0000FF'}}, rectangle={'shapeOptions': {'color': '#0000FF'}}),), layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7442756453842, 36.0722810745507], [-96.7098530104362, 36.067094653246], [-96.7093694958149, 36.0547137257744], [-96.7437479059881, 36.0598721686489]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7436085644239, 36.0613428157698], [-96.709155620014, 36.0561119936377], [-96.7086699104807, 36.0436792614937], [-96.7430781716137, 36.0488819455794]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7429782168562, 36.0505307943618], [-96.7084937215904, 36.0452556498693], [-96.7080056314503, 36.032770121514], [-96.7424449777545, 36.0380169646773]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7423328195289, 36.0396611737853], [-96.7078163640733, 36.034341994627], [-96.7073259579954, 36.0218031685162], [-96.7417967992815, 36.0270938820298]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7416420482302, 36.0283741162372], [-96.7070920079822, 36.0230097552664], [-96.7065991784154, 36.0104151207176], [-96.7411031197996, 36.0157508450781]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7408775352499, 36.0171670344327], [-96.7062932588319, 36.0117573935142], [-96.7057984972083, 35.999106174097], [-96.7403361941462, 36.0044870080873]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7400968740209, 36.0059050851025], [-96.7054778770653, 36.000450628053], [-96.7049804743866, 35.9877421388509], [-96.739552395163, 35.9931676115966]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7393598962686, 35.994705684624], [-96.7047051190916, 35.9892066719094], [-96.7042052533097, 35.9764400729692], [-96.7388124595093, 35.9819099243501]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7386460376711, 35.983414290999], [-96.7039540581817, 35.9778714556337], [-96.7034527573245, 35.9650458499153], [-96.738095698163, 35.9705597531476]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7380303389608, 35.9722555768997], [-96.7033007322483, 35.9666689845732], [-96.7027963762788, 35.9537836953055], [-96.7374766030122, 35.9593411114028]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7372135615813, 35.96091122663], [-96.7024471748597, 35.9552816450014], [-96.7019400854681, 35.9423365502839], [-96.7366567546088, 35.9479367135895]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7363918040281, 35.949307405904], [-96.7015861101954, 35.9436339918522], [-96.7010757789606, 35.9306258716076], [-96.7358314007149, 35.9362696137755]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7356733551492, 35.9377348201297], [-96.7008270804538, 35.9320168996439], [-96.7003137558651, 35.9189449218245], [-96.7351096011536, 35.9246329104424]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7350697228105, 35.9260170996656], [-96.7001824992082, 35.9202544340644], [-96.6996664471897, 35.907118451773], [-96.7345031867578, 35.9128507357414]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7343910287905, 35.9142946224681], [-96.6994622392658, 35.9084888337799], [-96.6989429512502, 35.8952877812917], [-96.7338208994473, 35.9010629363673]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7336409937305, 35.9026372644821], [-96.6986696514256, 35.8967880788772], [-96.6981473860089, 35.8835208540737], [-96.7330672187202, 35.88933931904]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates'

In [129]:
imgs = list(triplets['7Z']['d1'].values())
imgs.sort()
imgs

footprint = []
for i in tqdm(imgs):
    footprint.append(get_image_longlat_polygon(i))
    
m = ipytools.clickablemap()

for i in range(len(imgs)):
    m.add_GeoJSON(footprint[i])

m.center = footprint0[ 'coordinates'][0][0][::-1]
display(m)

100%|██████████| 18/18 [00:01<00:00,  9.97it/s]


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[36.0722810745507, -96.7442756453842], controls=(DrawControl(circle={'shapeOptions': {'color': '#0000FF'}}, layer=FeatureGroup(), polygon={'shapeOptions': {'color': '#0000FF'}}, polyline={'shapeOptions': {'color': '#0000FF'}}, rectangle={'shapeOptions': {'color': '#0000FF'}}),), layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7420042079892, 36.0774038791353], [-96.6997790610637, 36.0894952807075], [-96.6991694679827, 36.0759723765232], [-96.7414326311088, 36.0638225246905]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7413885320532, 36.065741624495], [-96.6993104406706, 36.0776825050229], [-96.6987043020469, 36.0642166282372], [-96.7408199366427, 36.0522180286217]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7406056925436, 36.0540864509041], [-96.6986749511422, 36.065877560692], [-96.6980721984383, 36.0524684495685], [-96.7400400170832, 36.0406203442384]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7398288407227, 36.0424490764106], [-96.6980446720197, 36.0540914364569], [-96.6974453185644, 36.040738741393], [-96.7392660906724, 36.02904010446]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7391600799127, 36.0311378625885], [-96.6975156591651, 36.0426385812534], [-96.6969195486153, 36.029339686573], [-96.7386001276721, 36.0177833690785]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7383679863457, 36.0199358475058], [-96.6968637158301, 36.0312954314902], [-96.6962706522742, 36.018050335751], [-96.7378106316709, 36.0066358285185]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7376464128451, 36.0086946592184], [-96.6962809343632, 36.0199143421758], [-96.6956910209671, 36.006722528381], [-96.7370917650818, 35.9954485874891]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7369894750275, 35.9973953975178], [-96.6957614155699, 36.0084765188497], [-96.6951747303603, 35.9953374420991], [-96.7364376157015, 35.9842027180454]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7362567383727, 35.9859971946375], [-96.695165565873, 35.9969410309641], [-96.6945821195406, 35.9838541551718], [-96.7357076828886, 35.9728573617827]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7356862519141, 35.9745514480916], [-96.694730033248, 35.9853598243041], [-96.6941497333953, 35.9723246085964], [-96.7351399113875, 35.9614639100216]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.735210323419, 35.9630367782543], [-96.6943875022232, 35.9737109852355], [-96.6938104996915, 35.9607268400368], [-96.7346668524906, 35.9500009348722]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7345872857944, 35.9516699143461], [-96.6938984701584, 35.9622102626612], [-96.693324533388, 35.9492772764772], [-96.734046451952, 35.938685851252]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7340964133005, 35.9402287088345], [-96.6935398084864, 35.9506366914494], [-96.6929690369825, 35.937754239541], [-96.7335583208996, 35.9272957906494]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7334017737458, 35.9289700578309], [-96.6929784352067, 35.9392457200211], [-96.6924105734561, 35.9264140103224], [-96.732866163186, 35.9160884917751]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7328011948905, 35.9176465600786], [-96.6925095447472, 35.9277911573348], [-96.6919447436044, 35.9150095838305], [-96.7322682233266, 35.9048157300914]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7321559635309, 35.9067241885868], [-96.6919908257018, 35.9167434605281], [-96.6914289355765, 35.9040098396586], [-96.7316254980158, 35.893941882582]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates':

In [130]:
imgs = list(triplets['8Z']['d1'].values())
imgs.sort()
imgs

footprint = []
for i in tqdm(imgs):
    footprint.append(get_image_longlat_polygon(i))
    
m = ipytools.clickablemap()

for i in range(len(imgs)):
    m.add_GeoJSON(footprint[i])

m.center = footprint0[ 'coordinates'][0][0][::-1]
display(m)

100%|██████████| 21/21 [00:02<00:00,  9.85it/s]


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[36.0722810745507, -96.7442756453842], controls=(DrawControl(circle={'shapeOptions': {'color': '#0000FF'}}, layer=FeatureGroup(), polygon={'shapeOptions': {'color': '#0000FF'}}, polyline={'shapeOptions': {'color': '#0000FF'}}, rectangle={'shapeOptions': {'color': '#0000FF'}}),), layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7444506607542, 36.0630813277031], [-96.710915579583, 36.0644312212303], [-96.7105193209155, 36.0544144796801], [-96.7440521904394, 36.053043341653]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7438327124788, 36.054232571186], [-96.7103531847947, 36.0555129418341], [-96.7099577502941, 36.0455062983971], [-96.7434347501331, 36.0442048112911]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7434069865466, 36.0453928907525], [-96.7099810580321, 36.0466040722434], [-96.7095865247295, 36.0366068857084], [-96.7430096096714, 36.0353747102562]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7429080157184, 36.0369544057404], [-96.7095351038615, 36.0380964402032], [-96.7091410365987, 36.0281083465126], [-96.7425107819575, 36.0269454389379]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7423406770175, 36.0278409839675], [-96.7090203452904, 36.0289153687826], [-96.7086272120223, 36.0189358438648], [-96.7419440665345, 36.0178407030089]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.741760975879, 36.019104854822], [-96.708490503908, 36.0201146943281], [-96.7080981616548, 36.0101429915787], [-96.7413648571444, 36.0091125059241]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7412297602473, 36.0104224929534], [-96.7080080699675, 36.0113682477017], [-96.7076163943803, 36.0014038749036], [-96.740834007931, 36.0004375805741]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7406955999893, 36.0015121740504], [-96.7075235787428, 36.0023912748909], [-96.7071326467794, 35.992434053222], [-96.7403002778574, 35.9915345203746]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7401389037441, 35.9927132846697], [-96.7070157763871, 35.9935263655693], [-96.7066252772157, 35.9835758365355], [-96.7397436983463, 35.9827424284294]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7396153455331, 35.9838838806417], [-96.7065399884079, 35.9846312011915], [-96.7061500583916, 35.9746868380522], [-96.7392203944837, 35.9739192914641]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7389556080908, 35.9749595883864], [-96.7059278494509, 35.975641486592], [-96.7055386765644, 35.965702916533], [-96.7385611018713, 35.9650008917097]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7384715749357, 35.9664175674124], [-96.7054879236499, 35.9670367095594], [-96.705099504655, 35.9571031062257], [-96.7380775244056, 35.9564639296532]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7378987671419, 35.9578545633911], [-96.7049588650267, 35.9584121803321], [-96.7045708118721, 35.948483180038], [-96.7375047785493, 35.9479056166872]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7374232303542, 35.94929911115], [-96.7045257378263, 35.9497953591912], [-96.7041381112628, 35.9398704115509], [-96.7370293668555, 35.9393543024373]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7369024366061, 35.9407813037919], [-96.7040467050894, 35.9412164049806], [-96.7036595213414, 35.9312951057474], [-96.7365087142845, 35.9308402265603]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7364025468739, 35.9323060713112], [-96.703587622938, 35.9326807693443], [-96.7032005828485, 35.9227626172427], [-96.7360086644386, 35.9223682213444]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates

In [132]:
imgs = list(triplets['7Z']['d2'].values())
imgs.sort()
imgs

footprint = []
for i in tqdm(imgs):
    footprint.append(get_image_longlat_polygon(i))
    
m = ipytools.clickablemap()

for i in range(len(imgs)):
    m.add_GeoJSON(footprint[i])

m.center = footprint0[ 'coordinates'][0][0][::-1]
display(m)

100%|██████████| 18/18 [00:01<00:00,  9.96it/s]


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[36.0722810745507, -96.7442756453842], controls=(DrawControl(circle={'shapeOptions': {'color': '#0000FF'}}, layer=FeatureGroup(), polygon={'shapeOptions': {'color': '#0000FF'}}, polyline={'shapeOptions': {'color': '#0000FF'}}, rectangle={'shapeOptions': {'color': '#0000FF'}}),), layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7811354160731, 36.1075931445452], [-96.7386751511896, 36.1195815632118], [-96.7380768798033, 36.1061109605398], [-96.7805680115124, 36.0940606921384]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7803800778979, 36.0958913076648], [-96.7380679719191, 36.1077297017747], [-96.737472847735, 36.0943155236429], [-96.7798153707801, 36.0824160044511]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7794574904837, 36.0841966639117], [-96.7372938597934, 36.0958857648881], [-96.7367018169547, 36.0825277705], [-96.7788954173308, 36.0707782625151]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7785415822501, 36.0725199766148], [-96.7365256205518, 36.0840608156169], [-96.7359366712575, 36.0707586443588], [-96.777982157464, 36.059158108723]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7777402314694, 36.0611711319593], [-96.7358650480087, 36.0725707969166], [-96.7352790508155, 36.0593218526485], [-96.777183334145, 36.0478631635101]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7768157999127, 36.0499310256884], [-96.7350817960679, 36.0611900150265], [-96.7344985538539, 36.0479943071792], [-96.7762612315652, 36.0366769631816]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7759627570064, 36.0386522384332], [-96.734368573343, 36.0497717833587], [-96.7337881915761, 36.0366287929153], [-96.7754106275121, 36.0254515523292]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7751753309678, 36.0273158691935], [-96.7337195730612, 36.0382973072746], [-96.7331421312088, 36.0252064875434], [-96.7746257237128, 36.0141680027699]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7743125465222, 36.0158810291237], [-96.7329946604432, 36.0267256280866], [-96.7324201722756, 36.0136864530595], [-96.7737654789879, 36.002785447836]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7736140023545, 36.0043986778438], [-96.7324320535022, 36.0151082511511], [-96.731860427593, 36.0021201884476], [-96.7730693872147, 35.9913548374198]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7730109584806, 35.9928480947916], [-96.7319633495528, 36.0034239397627], [-96.7313947378514, 35.9904863868674], [-96.7724689521739, 35.979855382399]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7722609640678, 35.9814447576355], [-96.7313483038018, 35.9918871853878], [-96.7307824749709, 35.9790002307967], [-96.7717213329782, 35.9685032598927]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7716444256353, 35.9699676619476], [-96.7308648980455, 35.9802781450087], [-96.7303019531661, 35.9674411827913], [-96.7711072755453, 35.9570767612908]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7708239812589, 35.9586724692578], [-96.7301786233504, 35.9688510713742], [-96.7296183054965, 35.9560642888846], [-96.7702890518127, 35.9458323529218]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7700985061297, 35.9473131527359], [-96.7295857572511, 35.9573611178755], [-96.7290282198048, 35.9446239171179], [-96.7695659562457, 35.9345232124159]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7693334017813, 35.9363565011599], [-96.728948036008, 35.9462795429615], [-96.7283931407909, 35.933589772878], [-96.768803109538, 35.9236145578182]]]}), GeoJSON(data={'type': 'Polygon', 'coordinate

In [134]:
type(triplets) == dict

True

In [131]:
imgs = list(triplets['8Z']['d2'].values())
imgs.sort()
imgs

footprint = []
for i in tqdm(imgs):
    footprint.append(get_image_longlat_polygon(i))
    
m = ipytools.clickablemap()

for i in range(len(imgs)):
    m.add_GeoJSON(footprint[i])

m.center = footprint0[ 'coordinates'][0][0][::-1]
display(m)

100%|██████████| 21/21 [00:02<00:00, 10.19it/s]


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[36.0722810745507, -96.7442756453842], controls=(DrawControl(circle={'shapeOptions': {'color': '#0000FF'}}, layer=FeatureGroup(), polygon={'shapeOptions': {'color': '#0000FF'}}, polyline={'shapeOptions': {'color': '#0000FF'}}, rectangle={'shapeOptions': {'color': '#0000FF'}}),), layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7753867740955, 36.0922134930637], [-96.741631380173, 36.0934777908426], [-96.7412157974791, 36.0834523542025], [-96.7749630970429, 36.0821627030972]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7747184246331, 36.0833965048915], [-96.7410184918645, 36.0845911607923], [-96.7406034733328, 36.0745752808472], [-96.7742950024563, 36.0733553898033]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7742438500235, 36.0745902181816], [-96.7405973844632, 36.0757155520438], [-96.7401830061956, 36.0657085932379], [-96.7738207584982, 36.0645581366946]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7736978730073, 36.0661862417086], [-96.7401042602394, 36.0672422916167], [-96.7396900789725, 36.0572438658106], [-96.773274661379, 36.0561628036363]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7730824567268, 36.0571075702518], [-96.7395412885412, 36.0580958322822], [-96.7391277838864, 36.0481054537106], [-96.7726596168236, 36.0470922862824]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7724574480455, 36.0484056960278], [-96.738966016171, 36.0493292791584], [-96.7385530539498, 36.0393461859494], [-96.7720348569702, 36.0383977974126]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7718823177173, 36.0397586100878], [-96.7384395060662, 36.040617964982], [-96.7380269598299, 36.0306416946574], [-96.7714598475651, 36.0297576308257]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7713032410236, 36.030886483387], [-96.7379099293849, 36.0316790270764], [-96.7374978639347, 36.0217093666384], [-96.7708809435692, 36.0208922108984]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7707033099282, 36.0221266308393], [-96.7373586871478, 36.0228529934027], [-96.7369467885878, 36.0128894802725], [-96.7702808682482, 36.0121385996264]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7701370762108, 36.0133376281965], [-96.7368400374386, 36.0139980611318], [-96.7364284426778, 36.0040401648595], [-96.7697146283437, 36.0033553040259]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7694342153465, 36.0044546172691], [-96.7361845851824, 36.0050494575385], [-96.7357734842633, 35.9950967867112], [-96.7690119533331, 35.9944776068343]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7689110826608, 35.9959525528874], [-96.735705356994, 35.9964849153712], [-96.7352945350557, 35.9865366753192], [-96.7684888031072, 35.9859800534683]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7682988057881, 35.9874323368854], [-96.7351366047472, 35.9879025454462], [-96.7347261123932, 35.9779583558561], [-96.7678765592821, 35.9774639661126]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7677857772144, 35.9789197628458], [-96.7346657343053, 35.9793284170317], [-96.7342554108159, 35.9693877613578], [-96.7673634018899, 35.96895500029]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7672279814459, 35.9704459678083], [-96.7341494675417, 35.9707932862571], [-96.7337393289996, 35.960855717559], [-96.7668054930942, 35.9604843639786]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7666928594023, 35.9620155232753], [-96.7336548860489, 35.9623022442858], [-96.7332446303422, 35.9523672960884], [-96.7662699537725, 35.9520566087872]]]}), GeoJSON(data={'type': 'Polygon', 'coord

In [142]:
geolocalisation(triplets['8Z']['d2'])

100%|██████████| 21/21 [00:02<00:00, 10.00it/s]


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[36.0922134930637, -96.7753867740955], controls=(DrawControl(circle={'shapeOptions': {'color': '#0000FF'}}, layer=FeatureGroup(), polygon={'shapeOptions': {'color': '#0000FF'}}, polyline={'shapeOptions': {'color': '#0000FF'}}, rectangle={'shapeOptions': {'color': '#0000FF'}}),), layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7753867740955, 36.0922134930637], [-96.741631380173, 36.0934777908426], [-96.7412157974791, 36.0834523542025], [-96.7749630970429, 36.0821627030972]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7747184246331, 36.0833965048915], [-96.7410184918645, 36.0845911607923], [-96.7406034733328, 36.0745752808472], [-96.7742950024563, 36.0733553898033]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7742438500235, 36.0745902181816], [-96.7405973844632, 36.0757155520438], [-96.7401830061956, 36.0657085932379], [-96.7738207584982, 36.0645581366946]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7736978730073, 36.0661862417086], [-96.7401042602394, 36.0672422916167], [-96.7396900789725, 36.0572438658106], [-96.773274661379, 36.0561628036363]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7730824567268, 36.0571075702518], [-96.7395412885412, 36.0580958322822], [-96.7391277838864, 36.0481054537106], [-96.7726596168236, 36.0470922862824]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7724574480455, 36.0484056960278], [-96.738966016171, 36.0493292791584], [-96.7385530539498, 36.0393461859494], [-96.7720348569702, 36.0383977974126]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7718823177173, 36.0397586100878], [-96.7384395060662, 36.040617964982], [-96.7380269598299, 36.0306416946574], [-96.7714598475651, 36.0297576308257]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7713032410236, 36.030886483387], [-96.7379099293849, 36.0316790270764], [-96.7374978639347, 36.0217093666384], [-96.7708809435692, 36.0208922108984]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7707033099282, 36.0221266308393], [-96.7373586871478, 36.0228529934027], [-96.7369467885878, 36.0128894802725], [-96.7702808682482, 36.0121385996264]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7701370762108, 36.0133376281965], [-96.7368400374386, 36.0139980611318], [-96.7364284426778, 36.0040401648595], [-96.7697146283437, 36.0033553040259]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7694342153465, 36.0044546172691], [-96.7361845851824, 36.0050494575385], [-96.7357734842633, 35.9950967867112], [-96.7690119533331, 35.9944776068343]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7689110826608, 35.9959525528874], [-96.735705356994, 35.9964849153712], [-96.7352945350557, 35.9865366753192], [-96.7684888031072, 35.9859800534683]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7682988057881, 35.9874323368854], [-96.7351366047472, 35.9879025454462], [-96.7347261123932, 35.9779583558561], [-96.7678765592821, 35.9774639661126]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7677857772144, 35.9789197628458], [-96.7346657343053, 35.9793284170317], [-96.7342554108159, 35.9693877613578], [-96.7673634018899, 35.96895500029]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7672279814459, 35.9704459678083], [-96.7341494675417, 35.9707932862571], [-96.7337393289996, 35.960855717559], [-96.7668054930942, 35.9604843639786]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7666928594023, 35.9620155232753], [-96.7336548860489, 35.9623022442858], [-96.7332446303422, 35.9523672960884], [-96.7662699537725, 35.9520566087872]]]}), GeoJSON(data={'type': 'Polygon', 'coord

In [144]:
imgs = list(triplets['7Z']['d2'].values())
imgs.sort()
imgs

footprint = []
for i in tqdm(imgs):
    footprint.append(get_image_longlat_polygon(i))
    
m = ipytools.clickablemap()

for i in range(len(imgs)):
    m.add_GeoJSON(footprint[i])

m.center = footprint0[ 'coordinates'][0][0][::-1]
display(m)

100%|██████████| 18/18 [00:01<00:00,  9.93it/s]


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[36.0722810745507, -96.7442756453842], controls=(DrawControl(circle={'shapeOptions': {'color': '#0000FF'}}, layer=FeatureGroup(), polygon={'shapeOptions': {'color': '#0000FF'}}, polyline={'shapeOptions': {'color': '#0000FF'}}, rectangle={'shapeOptions': {'color': '#0000FF'}}),), layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7811354160731, 36.1075931445452], [-96.7386751511896, 36.1195815632118], [-96.7380768798033, 36.1061109605398], [-96.7805680115124, 36.0940606921384]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7803800778979, 36.0958913076648], [-96.7380679719191, 36.1077297017747], [-96.737472847735, 36.0943155236429], [-96.7798153707801, 36.0824160044511]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7794574904837, 36.0841966639117], [-96.7372938597934, 36.0958857648881], [-96.7367018169547, 36.0825277705], [-96.7788954173308, 36.0707782625151]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7785415822501, 36.0725199766148], [-96.7365256205518, 36.0840608156169], [-96.7359366712575, 36.0707586443588], [-96.777982157464, 36.059158108723]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7777402314694, 36.0611711319593], [-96.7358650480087, 36.0725707969166], [-96.7352790508155, 36.0593218526485], [-96.777183334145, 36.0478631635101]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7768157999127, 36.0499310256884], [-96.7350817960679, 36.0611900150265], [-96.7344985538539, 36.0479943071792], [-96.7762612315652, 36.0366769631816]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7759627570064, 36.0386522384332], [-96.734368573343, 36.0497717833587], [-96.7337881915761, 36.0366287929153], [-96.7754106275121, 36.0254515523292]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7751753309678, 36.0273158691935], [-96.7337195730612, 36.0382973072746], [-96.7331421312088, 36.0252064875434], [-96.7746257237128, 36.0141680027699]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7743125465222, 36.0158810291237], [-96.7329946604432, 36.0267256280866], [-96.7324201722756, 36.0136864530595], [-96.7737654789879, 36.002785447836]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7736140023545, 36.0043986778438], [-96.7324320535022, 36.0151082511511], [-96.731860427593, 36.0021201884476], [-96.7730693872147, 35.9913548374198]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7730109584806, 35.9928480947916], [-96.7319633495528, 36.0034239397627], [-96.7313947378514, 35.9904863868674], [-96.7724689521739, 35.979855382399]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7722609640678, 35.9814447576355], [-96.7313483038018, 35.9918871853878], [-96.7307824749709, 35.9790002307967], [-96.7717213329782, 35.9685032598927]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7716444256353, 35.9699676619476], [-96.7308648980455, 35.9802781450087], [-96.7303019531661, 35.9674411827913], [-96.7711072755453, 35.9570767612908]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7708239812589, 35.9586724692578], [-96.7301786233504, 35.9688510713742], [-96.7296183054965, 35.9560642888846], [-96.7702890518127, 35.9458323529218]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7700985061297, 35.9473131527359], [-96.7295857572511, 35.9573611178755], [-96.7290282198048, 35.9446239171179], [-96.7695659562457, 35.9345232124159]]]}), GeoJSON(data={'type': 'Polygon', 'coordinates': [[[-96.7693334017813, 35.9363565011599], [-96.728948036008, 35.9462795429615], [-96.7283931407909, 35.933589772878], [-96.768803109538, 35.9236145578182]]]}), GeoJSON(data={'type': 'Polygon', 'coordinate

In [145]:
triplets

{'1Z': {'d1': {1: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0001.tif',
   2: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0002.tif',
   3: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0003.tif',
   4: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0004.tif',
   5: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0005.tif',
   6: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0006.tif',
   7: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0007.tif',
   8: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0008.tif',
   9: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0009.tif',
   10: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0010.tif',
   11: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d1_0011.tif',
   12: '../data/s0

In [150]:
triplets['1Z'].values()

dict_values([{8: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0008.tif', 2: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0002.tif', 18: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0018.tif', 10: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0010.tif', 11: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0011.tif', 4: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0004.tif', 7: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0007.tif', 6: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0006.tif', 12: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0012.tif', 5: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0005.tif', 17: '../data/s03_20161003T161231Z/panchromatic/s03_20161003T161231Z_pan_d2_0017.tif', 1: '../data/s03_20161003T161231Z/panchromatic/